## Which jobs grew in pay after adjusting for inflation. 

In [1]:
import sqlite3
import os
import pandas as pd

In [2]:
occupation_file_name = os.path.join('Occupation_Data.csv')

In [3]:
occupation = pd.read_csv(occupation_file_name, index_col=None)

In [4]:
occupation.head()

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR
0,00-0000,All Occupations,total,652100.0,1.3,21.99,45740.0,1.5,9.18,11.90,17.47,26.74,38.71,19100.0,24760.0,36350.0,55610.0,80510.0,2018
1,11-0000,Management Occupations,major,31180.0,2.6,49.67,103320.0,1.5,19.90,28.60,42.37,61.46,87.13,41400.0,59490.0,88120.0,127850.0,181230.0,2018
2,11-1011,Chief Executives,detailed,1210.0,6.1,73.53,152940.0,2.8,28.36,41.72,61.92,95.40,NaN,58980.0,86770.0,128800.0,198420.0,NaN,2018
3,11-1021,General and Operations Managers,detailed,10360.0,4.4,48.99,101900.0,2.1,17.27,24.89,38.13,61.21,96.22,35920.0,51770.0,79310.0,127320.0,200130.0,2018
4,11-2011,Advertising and Promotions Managers,detailed,60.0,16.8,53.04,110320.0,10.4,18.64,26.74,44.61,78.86,98.10,38770.0,55630.0,92780.0,164030.0,204050.0,2018


In economics the term real dollars is the value of currency after being adjusted for inflation. I will turn everything into 2018 dollars and to compute the difference I used an inflation calculator which can be found in the following website 
https://www.usinflationcalculator.com/

Adjusted for inflation, 1.00 in 2003 is equal to 1.36 in 2018, with a 36.5% cumulative rate of inflation.

Adjusted for inflation, 1.00 in 2008 is equal to 1.17 in 2018, with a 16.6% cumulative rate of inflation.

Adjusted for inflation, 1.00 in 2013 is equal to 1.08 in 2018, with a 7.8% cumulative rate of inflation.

The following sections will need to be adjusted based on their year. 
H_PCT10, H_PCT25, H_MEDIAN, H_CT75, H_PCT90, A_PCT10, A_PCT25, A_MEDIAN, A_PCT75, A_PCT90

In [5]:
year2018 = occupation['YEAR'] == 2018
year2013 = occupation['YEAR'] == 2013
year2008 = occupation['YEAR'] == 2008
year2003 = occupation['YEAR'] == 2003

## Converting to 2018 dollars

In [6]:
occupation.columns[5:18]

for x in occupation.columns[5:18]:
    occupation.loc[(occupation.YEAR == 2013), x] *=1.08
    occupation.loc[(occupation.YEAR == 2008), x] *=1.17
    occupation.loc[(occupation.YEAR == 2003), x] *=1.36


## Checking to see how many occupations existed across all four sample years

In [7]:
occ_count=occupation.groupby('OCC_CODE')
occ_count_all = occ_count.size()
occ_count_all.value_counts()

4    384
2    139
3    128
1     80
5      5
dtype: int64

## Found an error in the data I need to find out where it is and remove it. (Skip to next section if you wish to skip this)
It is odd that there is a 5 in the options I need to figure out which year(s) have duplicate occupations.

In [8]:
dupe_year=occupation.groupby(['OCC_CODE', 'YEAR']).size() > 1
print(dupe_year.value_counts())
dupe_year_list=dupe_year[dupe_year == True]
dupe_year_list

False    2293
True        5
dtype: int64


OCC_CODE  YEAR
53-7061   2003    True
53-7062   2003    True
53-7063   2003    True
53-7064   2003    True
53-7081   2003    True
dtype: bool

2003 Seems to have produced all of the duplicates and here is the list of duplicate job codes 

I am going to have to remove the duplicate data from the dataset

In [9]:
dupes_year = occupation[occupation.duplicated(('OCC_CODE', 'YEAR'), keep=False)]

In [10]:
dupes_year.sort_values('OCC_CODE')

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR
2292,53-7061,Cleaners of vehicles and equipment,NaN,1260.0,15.7,13.1784,27404.0,6.664,8.1192,9.3432,11.5736,15.3408,19.6656,16891.2,19434.4,24085.6,31905.6,40908.8,2003
2298,53-7061,Cleaners of vehicles and equipment,NaN,1320.0,12.4,13.9264,28954.4,10.336,8.0784,9.2344,11.5464,16.1160,24.1672,16796.0,19203.2,24017.6,33524.0,50252.0,2003
2293,53-7062,"Laborers and freight, stock, and material move...",NaN,12700.0,5.6,15.3000,31824.0,6.120,9.5608,11.2200,13.4912,16.4016,26.5336,19883.2,23337.6,28070.4,34122.4,55188.8,2003
2299,53-7062,"Laborers and freight, stock, and material move...",NaN,12600.0,9.3,15.6672,32572.0,5.304,9.5336,11.4512,14.0080,17.9520,26.1800,19842.4,23813.6,29131.2,37345.6,54454.4,2003
2294,53-7063,Machine feeders and offbearers,NaN,1650.0,29.6,17.0952,35550.4,5.848,11.0432,13.0016,14.9736,20.4136,24.3304,22956.8,27036.8,31130.4,42445.6,50605.6,2003
2300,53-7063,Machine feeders and offbearers,NaN,1410.0,29.9,17.0680,35509.6,6.936,10.7848,12.6888,14.8512,20.4272,24.2624,22426.4,26397.6,30885.6,42486.4,50456.0,2003
2295,53-7064,"Packers and packagers, hand",NaN,4970.0,11.3,11.9952,24956.0,2.312,8.2008,9.4248,11.3832,13.9264,17.5168,17054.4,19611.2,23691.2,28968.0,36420.8,2003
2301,53-7064,"Packers and packagers, hand",NaN,4650.0,9.0,12.1448,25255.2,3.128,8.2280,9.3432,11.3016,14.1984,18.2784,17095.2,19434.4,23514.4,29525.6,38025.6,2003
2296,53-7081,Refuse and recyclable material collectors,NaN,400.0,29.9,16.1976,33687.2,6.936,10.4992,12.8520,15.9120,18.8088,22.6032,21855.2,26724.0,33088.8,39127.2,47001.6,2003
2302,53-7081,Refuse and recyclable material collectors,NaN,420.0,29.0,15.7488,32748.8,8.024,10.7032,12.2672,14.5384,18.4824,22.1816,22249.6,25513.6,30232.8,38460.8,46131.2,2003


Much to my suprise the data is different. I suppose data gathering methods were not refined in 2003. To correct this I will merge the data and take the average. Once I get the averages I will add this to a new data base and remove all other duplicates. 

In [11]:
#Had to change the as_index to False so it would create number indexs rather than use the OOC_code
combine = dupes_year.groupby(['OCC_CODE', 'OCC_TITLE', 'YEAR'], as_index=False).mean()
combine = combine.copy()

In [12]:
combine['OCC_GROUP']=pd.np.NaN

In [13]:
new_occupation= occupation.append(combine, ignore_index=True, sort=False)

In [14]:
new_occupation.drop_duplicates(('OCC_CODE', 'YEAR'), keep='last', inplace=True)

In [15]:
occ_count=new_occupation.groupby('OCC_CODE')
occ_count_all = occ_count.size()
occ_count_4=occ_count.size() == 4
occ_count_3=occ_count.size() == 3
occ_count_2=occ_count.size() == 2
occ_count_1=occ_count.size() == 1

In [16]:
print(occ_count_all.value_counts())

4    389
2    139
3    128
1     80
dtype: int64


### Problem Solved
The problem has been solved. I know this becasue the 5 group no longer exists and the 4 group increased in size by five (the same number as the old 5 group). 
There are 736 unique occupations on this list of those occupations 389 occupations existed all four observed years, 128 existed for three observed years, 139 exitsed for two observed years, and 80 existed for only one observed year. 

In [17]:
#made dictionaries that can be used to show years
total_instance=new_occupation.groupby(['OCC_CODE'])
total_instance=total_instance.size()
dict4={}
dict3={}
dict2={}
dict1={}
for y, x in total_instance.items():
    if x == 4:
        dict4.update({y:x})
    elif x == 3:
        dict3.update({y:x})
    elif x == 2:
        dict2.update({y:x})
    else:
        dict1.update({y:x})   

In [18]:
check=(new_occupation.groupby(['OCC_CODE','YEAR'])).size()

In [19]:
occ_yeardict={}
for y,x in check.items():
    (a,b)=y
    if a in occ_yeardict:
        t=occ_yeardict[a]
        t = t + (b,)
        occ_yeardict.update({a:t})
    else:
        occ_yeardict.update({a:(b,)})
    

In [20]:
occ_yeardict

{'00-0000': (2003, 2008, 2013, 2018),
 '11-0000': (2003, 2008, 2013, 2018),
 '11-1011': (2003, 2008, 2013, 2018),
 '11-1021': (2003, 2008, 2013, 2018),
 '11-1031': (2003, 2008, 2013),
 '11-2011': (2003, 2008, 2013, 2018),
 '11-2021': (2003, 2008, 2013, 2018),
 '11-2022': (2003, 2008, 2013, 2018),
 '11-2031': (2003, 2008, 2013, 2018),
 '11-3011': (2003, 2008, 2013, 2018),
 '11-3021': (2003, 2008, 2013, 2018),
 '11-3031': (2008, 2013, 2018),
 '11-3040': (2003,),
 '11-3041': (2008,),
 '11-3042': (2008,),
 '11-3049': (2008,),
 '11-3051': (2003, 2008, 2013, 2018),
 '11-3061': (2003, 2008, 2013, 2018),
 '11-3071': (2003, 2008, 2013, 2018),
 '11-3111': (2013, 2018),
 '11-3121': (2013, 2018),
 '11-3131': (2013, 2018),
 '11-9021': (2003, 2008, 2013, 2018),
 '11-9031': (2003, 2008, 2013, 2018),
 '11-9032': (2003, 2008, 2013, 2018),
 '11-9033': (2008, 2013, 2018),
 '11-9039': (2008, 2013, 2018),
 '11-9041': (2003, 2008, 2013, 2018),
 '11-9051': (2003, 2008, 2013, 2018),
 '11-9061': (2003, 2008, 2

In [21]:
dictgap={}
for x, y in occ_yeardict.items():
    z=pd.np.diff(y)
    if (len(z)>0) and ((10 in z)or(15 in z)):
        if z[0]>5:
            dictgap.update({x:(y[0:2],z[0])})
        elif z[1]>5:
            dictgap.update({x:(y[1:],z[1])})
            
dictgap


{'13-1011': ((2003, 2018), 15),
 '19-1012': ((2008, 2018), 10),
 '19-1029': ((2008, 2018), 10),
 '19-2021': ((2008, 2018), 10),
 '19-2042': ((2003, 2013), 10),
 '21-1029': ((2008, 2018), 10),
 '23-1023': ((2003, 2013), 10),
 '25-1054': ((2003, 2013), 10),
 '25-2032': ((2008, 2018), 10),
 '25-9021': ((2008, 2018), 10),
 '27-1013': ((2008, 2018), 10),
 '27-1014': ((2003, 2018), 15),
 '27-1027': ((2003, 2013), 10),
 '27-2021': ((2008, 2018), 10),
 '27-4014': ((2003, 2013), 10),
 '27-4031': ((2008, 2018), 10),
 '29-2051': ((2008, 2018), 10),
 '29-2053': ((2003, 2018), 15),
 '39-5011': ((2003, 2018), 15),
 '39-6012': ((2003, 2013), 10),
 '41-9021': ((2003, 2018), 15),
 '41-9091': ((2003, 2013), 10),
 '45-2011': ((2008, 2018), 10),
 '45-2041': ((2008, 2018), 10),
 '47-2011': ((2008, 2018), 10),
 '47-2082': ((2008, 2018), 10),
 '47-4021': ((2003, 2013), 10),
 '49-2021': ((2003, 2013), 10),
 '49-9031': ((2003, 2013), 10),
 '49-9063': ((2008, 2018), 10),
 '51-4012': ((2008, 2018), 10),
 '51-405

In [22]:
len(dictgap)

43

In [23]:
new_occupation.sort_values(['YEAR', 'OCC_CODE'], ascending=[1, 1], inplace=True)
new_occupation

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR
1797,00-0000,All Occupations,NaN,542460.0,0.9,22.2360,46253.6,1.496,9.7376,12.6208,17.8296,27.4584,38.3520,20236.8,26261.6,37100.8,57120.0,79777.6,2003
1798,11-0000,Management occupations,major,28090.0,2.3,48.7152,101333.6,1.088,21.0664,28.8864,41.9560,60.0168,85.6936,43819.2,60084.8,87284.8,124820.8,178255.2,2003
1799,11-1011,Chief executives,NaN,2070.0,4.8,88.6856,184470.4,2.992,38.8824,54.5632,83.5448,NaN,NaN,80879.2,113492.0,173767.2,NaN,NaN,2003
1800,11-1021,General and operations managers,NaN,8730.0,2.7,48.8376,101592.0,1.904,20.8488,28.7232,41.8744,59.7856,87.0264,43370.4,59744.8,87094.4,124344.8,181016.0,2003
1801,11-1031,Legislators,NaN,90.0,28.3,21.2704,44254.4,13.056,8.2552,9.7376,22.2224,24.8200,30.1920,17176.8,20250.4,46226.4,51625.6,62804.8,2003
1802,11-2011,Advertising and promotions managers,NaN,330.0,10.6,36.0944,75072.0,6.256,20.1688,23.8680,29.3624,42.0512,61.7984,41942.4,49640.0,61077.6,87461.6,128547.2,2003
1803,11-2021,Marketing managers,NaN,710.0,8.2,53.9920,112295.2,4.080,26.0440,34.5848,47.3960,67.6464,91.9904,54168.8,71944.0,98586.4,140719.2,191338.4,2003
1804,11-2022,Sales managers,NaN,1470.0,7.8,51.5576,107222.4,4.760,24.3440,33.4832,45.6144,63.8928,83.4088,50632.8,69659.2,94873.6,132899.2,173495.2,2003
1805,11-2031,Public relations managers,NaN,210.0,10.1,40.0384,83286.4,4.216,23.0520,30.0968,37.3592,45.9544,62.3832,47940.0,62587.2,77710.4,95594.4,129757.6,2003
1806,11-3011,Administrative services managers,NaN,960.0,9.1,33.9592,70624.8,2.720,18.3192,22.9432,30.0832,40.9088,56.0048,38093.6,47722.4,62587.2,85095.2,116497.6,2003


In [31]:
new_occupation['H_CHANGE'] = new_occupation.groupby('OCC_CODE').H_MEAN.pct_change()
new_occupation

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,...,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR,H_CHANGE,H_Average
1797,00-0000,All Occupations,NaN,542460.0,0.9,22.2360,46253.6,1.496,9.7376,12.6208,...,27.4584,38.3520,20236.8,26261.6,37100.8,57120.0,79777.6,2003,NaN,NaN
1798,11-0000,Management occupations,major,28090.0,2.3,48.7152,101333.6,1.088,21.0664,28.8864,...,60.0168,85.6936,43819.2,60084.8,87284.8,124820.8,178255.2,2003,NaN,NaN
1799,11-1011,Chief executives,NaN,2070.0,4.8,88.6856,184470.4,2.992,38.8824,54.5632,...,NaN,NaN,80879.2,113492.0,173767.2,NaN,NaN,2003,NaN,NaN
1800,11-1021,General and operations managers,NaN,8730.0,2.7,48.8376,101592.0,1.904,20.8488,28.7232,...,59.7856,87.0264,43370.4,59744.8,87094.4,124344.8,181016.0,2003,NaN,NaN
1801,11-1031,Legislators,NaN,90.0,28.3,21.2704,44254.4,13.056,8.2552,9.7376,...,24.8200,30.1920,17176.8,20250.4,46226.4,51625.6,62804.8,2003,NaN,NaN
1802,11-2011,Advertising and promotions managers,NaN,330.0,10.6,36.0944,75072.0,6.256,20.1688,23.8680,...,42.0512,61.7984,41942.4,49640.0,61077.6,87461.6,128547.2,2003,NaN,NaN
1803,11-2021,Marketing managers,NaN,710.0,8.2,53.9920,112295.2,4.080,26.0440,34.5848,...,67.6464,91.9904,54168.8,71944.0,98586.4,140719.2,191338.4,2003,NaN,NaN
1804,11-2022,Sales managers,NaN,1470.0,7.8,51.5576,107222.4,4.760,24.3440,33.4832,...,63.8928,83.4088,50632.8,69659.2,94873.6,132899.2,173495.2,2003,NaN,NaN
1805,11-2031,Public relations managers,NaN,210.0,10.1,40.0384,83286.4,4.216,23.0520,30.0968,...,45.9544,62.3832,47940.0,62587.2,77710.4,95594.4,129757.6,2003,NaN,NaN
1806,11-3011,Administrative services managers,NaN,960.0,9.1,33.9592,70624.8,2.720,18.3192,22.9432,...,40.9088,56.0048,38093.6,47722.4,62587.2,85095.2,116497.6,2003,NaN,NaN


In [39]:
#new_occupation['YEAR'] = pd.to_datetime(new_occupation['YEAR'], format = "%Y")

In [40]:
new_occupation.to_sql("occ_table",sqlite3.connect('occ.db'), if_exists ="replace")

In [41]:
con = sqlite3.connect("occ.db")

In [48]:
pd.read_sql_query("SELECT OCC_CODE,OCC_TITLE,AVG(H_CHANGE) as average FROM occ_table GROUP BY OCC_CODE ORDER BY average DESC", con)

,OCC_CODE,OCC_TITLE,average
0,27-3011,Radio and television announcers,1.131046
1,19-1023,Zoologists and Wildlife Biologists,0.899938
2,19-1042,"Medical Scientists, Except Epidemiologists",0.763306
3,21-1029,"Social workers, all other",0.521575
4,29-2091,Orthotists and Prosthetists,0.419233
5,15-1134,Web Developers,0.412409
6,29-1121,Audiologists,0.402714
7,13-1011,"Agents and business managers of artists, perfo...",0.396526
8,27-2042,Musicians and singers,0.355757
9,13-1072,"Compensation, benefits, and job analysis speci...",0.347175


In [47]:
pd.read_sql_query("SELECT * FROM occ_table WHERE OCC_CODE ='19-1023'", con)

,index,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,...,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,YEAR,H_CHANGE,H_Average
0,691,19-1023,Zoologists and Wildlife Biologists,detailed,NaN,NaN,18.7164,38923.2,4.536,16.146,...,19.062,19.7316,33577.2,34981.2,37314.0,39646.8,41050.8,2013,NaN,None
1,107,19-1023,Zoologists and Wildlife Biologists,detailed,60.0,47.0,35.5600,73960.0,5.400,21.510,...,38.410,68.0700,44750.0,52760.0,66820.0,79880.0,141590.0,2018,0.899938,None


In [29]:
occupation_2018 = pd.read_sql_query("SELECT * FROM occ_table WHERE YEAR=2018", con)
occupation_2013 = pd.read_sql_query("SELECT * FROM occ_table WHERE YEAR=2013", con)
occupation_2008 = pd.read_sql_query("SELECT * FROM occ_table WHERE YEAR=2008", con)
occupation_2003 = pd.read_sql_query("SELECT * FROM occ_table WHERE YEAR=2003", con)
type(occupation_2018)

pandas.core.frame.DataFrame

In [30]:
occ_yeardict

for x, y in occ_yeardict.items():
    years_check=len(y)
    if x not in dictgap:
        if years_check == 4:
            (y) = (y_1, y_2, y_3, y_4)
        elif years_check == 3:
        elif years_check == 2:
        else:

IndentationError: expected an indented block (<ipython-input-30-9726873ec765>, line 9)